```sql
select
    platformgameid,
    min(eventtime) as eventtime,
    min(gametime) as gametime,
    participants[1].shutdownvalue as shutdown_held_1,
    participants[2].shutdownvalue as shutdown_held_2,
    participants[3].shutdownvalue as shutdown_held_3,
    participants[4].shutdownvalue as shutdown_held_4,
    participants[5].shutdownvalue as shutdown_held_5,
    participants[6].shutdownvalue as shutdown_held_6,
    participants[7].shutdownvalue as shutdown_held_7,
    participants[8].shutdownvalue as shutdown_held_8,
    participants[9].shutdownvalue as shutdown_held_9,
    participants[10].shutdownvalue as shutdown_held_10
from lol.games
where
    eventtype = 'stats_update' and 
    participants[1].shutdownvalue > 0
    and CARDINALITY(participants) > 10
group by
platformgameid,
participants[1].shutdownvalue,
participants[2].shutdownvalue,
participants[3].shutdownvalue,
participants[4].shutdownvalue,
participants[5].shutdownvalue,
participants[6].shutdownvalue,
participants[7].shutdownvalue,
participants[8].shutdownvalue,
participants[9].shutdownvalue,
participants[10].shutdownvalue
```

In [15]:
import pandas as pd

shutdown = pd.read_csv('csv/sql/shutdown.csv')
shutdown.sort_values(by=["platformgameid", "gametime"], inplace=True)
shutdown.head()

,platformgameid,eventtime,gametime,shutdown_held_1,shutdown_held_2,shutdown_held_3,shutdown_held_4,shutdown_held_5,shutdown_held_6,shutdown_held_7,shutdown_held_8,shutdown_held_9,shutdown_held_10
326292,ESPORTSTMNT01:1310842,2020-03-13T05:30:17.724Z,705490,50.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65529,ESPORTSTMNT01:1310842,2020-03-13T05:31:46.811Z,794576,50.0,0.0,100.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0
65530,ESPORTSTMNT01:1310842,2020-03-13T05:32:05.244Z,813008,50.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36108,ESPORTSTMNT01:1310842,2020-03-13T05:33:24.093Z,891854,50.0,0.0,200.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0
208900,ESPORTSTMNT01:1310842,2020-03-13T05:33:32.285Z,900047,50.0,0.0,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df = shutdown
df.head()
df["platformgameid"].value_counts()
print(len(df["platformgameid"].unique()))

new_row = {
    "platformgameid": None,
    "gametime": None,
    "shutdown_held_1": None,
    "shutdown_held_2": None,
    "shutdown_held_3": None,
    "shutdown_held_4": None,
    "shutdown_held_5": None,
    "shutdown_held_6": None,
    "shutdown_held_7": None,
    "shutdown_held_8": None,
    "shutdown_held_9": None,
    "shutdown_held_10": None,
}

new = []
prev_row = None
for _, row in shutdown.iterrows():
    if prev_row is None:
        prev_row = row
        new_row["platformgameid"] = row["platformgameid"]
        new_row["gametime"] = row["gametime"]
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
        continue
    elif row["platformgameid"] != prev_row["platformgameid"]:
        new.append(new_row)
        new_row = {
            "platformgameid": row["platformgameid"],
            "gametime": row["gametime"],
        }
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
    elif row["platformgameid"] == new_row["platformgameid"]:
        for i in range(1, 11):
            if row["shutdown_held_" + str(i)] > prev_row["shutdown_held_" + str(i)]:
                new_row["shutdown_held_" + str(i)] += row["shutdown_held_" + str(i)]- prev_row["shutdown_held_" + str(i)] 


    prev_row = row

df = pd.DataFrame(new)
print(df)

16901
              platformgameid  gametime  shutdown_held_1  shutdown_held_2  \
0      ESPORTSTMNT01:1310842    705490            500.0            150.0   
1      ESPORTSTMNT01:1310866   1056752            600.0              0.0   
2      ESPORTSTMNT01:1310892    917488            250.0              0.0   
3      ESPORTSTMNT01:1313641   1367952            100.0              0.0   
4      ESPORTSTMNT01:1313662   1574820            300.0            500.0   
...                      ...       ...              ...              ...   
16895           LPL_B:329904    351030             50.0              0.0   
16896           LPL_B:329939    725032            300.0            500.0   
16897           LPL_B:329985    547021            250.0            200.0   
16898           LPL_B:330027    746019            450.0            150.0   
16899           LPL_B:330033   2206028            150.0              0.0   

       shutdown_held_3  shutdown_held_4  shutdown_held_5  shutdown_held_6  \
0   

In [37]:
shutdown_collected = pd.read_csv('shutdown_collected.csv')
shutdown_collected.fillna(0, inplace=True)
shutdown_collected.sort_values(by=["platformgameid"], inplace=True)

blue_side = shutdown_collected[shutdown_collected["side"] == 100]
blue_side.rename(columns={
    "teamid": "blue_teamid",
    "shutdownbounty": "blue_shutdown_collected",
}, inplace=True)
blue_side.drop(columns=["side"], inplace=True)
red_side = shutdown_collected[shutdown_collected["side"] == 200]
red_side.rename(columns={
    "teamid": "red_teamid",
    "shutdownbounty": "red_shutdown_collected",
}, inplace=True)
red_side.drop(columns=[
    "side",
], inplace=True)

shutdown = blue_side.merge(red_side, on="platformgameid")
blue_team_shutdown = df["shutdown_held_1"] + df["shutdown_held_2"] + df["shutdown_held_3"] + df["shutdown_held_4"] + df["shutdown_held_5"]
red_team_shutdown = df["shutdown_held_6"] + df["shutdown_held_7"] + df["shutdown_held_8"] + df["shutdown_held_9"] + df["shutdown_held_10"]
shutdown["blue_team_shutdown"] = blue_team_shutdown
shutdown["red_team_shutdown"] = red_team_shutdown
display(shutdown)

,platformgameid,teamid,side,shutdownbounty
43372,ESPORTSTMNT01:1110148,9.956641e+16,200,0.0
2942,ESPORTSTMNT01:1110148,1.007832e+17,100,0.0
46462,ESPORTSTMNT01:1110162,9.876799e+16,100,0.0
6208,ESPORTSTMNT01:1110162,9.956641e+16,200,0.0
47456,ESPORTSTMNT01:1110177,1.007832e+17,100,0.0


C:\Users\Jan\AppData\Local\Temp\ipykernel_14132\3733679433.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.rename(columns={
C:\Users\Jan\AppData\Local\Temp\ipykernel_14132\3733679433.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.drop(columns=["side"], inplace=True)
C:\Users\Jan\AppData\Local\Temp\ipykernel_14132\3733679433.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_side.rename(columns={
C:\U

,platformgameid,blue_teamid,blue_shutdown_collected,red_teamid,red_shutdown_collected,blue_team_shutdown,red_team_shutdown
0,ESPORTSTMNT01:1110148,1.007832e+17,0.0,9.956641e+16,0.0,2150.0,0.0
1,ESPORTSTMNT01:1110162,9.876799e+16,0.0,9.956641e+16,0.0,2750.0,1250.0
2,ESPORTSTMNT01:1110177,1.007832e+17,0.0,9.956641e+16,0.0,900.0,250.0
3,ESPORTSTMNT01:1110187,9.876799e+16,0.0,9.956641e+16,0.0,650.0,1250.0
4,ESPORTSTMNT01:1110198,9.956641e+16,0.0,9.956641e+16,0.0,2100.0,100.0
...,...,...,...,...,...,...,...
25010,LPL_B:330027,9.876799e+16,400.0,9.956640e+16,500.0,NaN,NaN
25011,LPL_B:330032,9.956640e+16,100.0,9.876799e+16,800.0,NaN,NaN
25012,LPL_B:330033,9.956640e+16,1300.0,9.876799e+16,1600.0,NaN,NaN
25013,LPL_B:330039,9.876799e+16,0.0,9.956640e+16,0.0,NaN,NaN
